# Quantum Tomography Overview

* **Author:** Gadi Aleksandrowicz (gadia@il.ibm.com)
* **Last Updated:** January 24, 2019



In [1]:
import numpy as np

## The General Theory

The goal of quantum tomography is obtain data on quantum systems by performing several different measurements. In Qiskit Ingis we are currently concerened with the following two tomography tasks:

1. **Quantum state tomograhpy**: Given a quantum state described by a density operator $\rho$, obtain a matrix representation for $\rho$.
2. **Quantum process tomograhpy**: Given a quantum channel $\mathcal{E}$, obtain a matrix representation (the **Choi matrix**) for $\mathcal{E}$.

In both cases we rely on the assumption that we have access to a large number of identical copies of the system and so can peform several different measures on it.

We can roughly split the tomography process to three stages:
1. Preperation: Add suitable initialization/measurement devices to the quantum system.
2. Experiment: Obtain measurement data from the quantum system.
3. Tomography: Use the obtained data to reconstruct the system's description.

Steps 1 and 2 are related to the quantum system being studied, whereas step 3 is a classical computation which can be carried out on standard computers.

## State Tomography Overview

Let $\rho$ be the density operator of a quantum state. If $\rho$ can be reliably reproduced, it can be subjected to several measurements with respect to different operators; this data can be used to reconstruct $\rho$ or a close approximation for it in several different methods.

### Example: 1-qubit reconstruction using the Pauli basis

Given the Pauli matrices $
I=\left(\begin{array}{cc}
1 & 0\\
0 & 1
\end{array}\right),
X=\left(\begin{array}{cc}
0 & 1\\
1 & 0
\end{array}\right),
Y=\left(\begin{array}{cc}
0 & -i\\
i & 0
\end{array}\right),
Z=\left(\begin{array}{cc}
1 & 0\\
0 & -1
\end{array}\right)$


In [3]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

It is easy to see they constitute an orthonormal basis for $M_2(\mathbb{C})$ with respect to the Hilbert-Schmidt inner product $\left\langle A,B\right\rangle =\frac{1}{2}\text{tr}\left(B^{\dagger}A\right)$

In [4]:
def HS_product(A,B):
    return np.trace(np.conj(B).T @ A)


And hence,

$$ \rho =\left\langle \rho,I\right\rangle I+\left\langle \rho,X\right\rangle X+\left\langle \rho,Y\right\rangle Y+\left\langle \rho,Z\right\rangle Z = $$

$$=\frac{\text{tr}\left(\rho\right)I+\text{tr}\left(X\rho\right)X+\text{tr}\left(Y\rho\right)Y+\text{tr}\left(Z\rho\right)Z}{2}$$

The values of $\text{tr}\left(X\rho\right), \text{tr}\left(Y\rho\right), \text{tr}\left(Z\rho\right)$ can be approximated by repeated measuring in the $X, Y$ and $Z$ bases. Since $\text{tr}\left(\rho\right)=1$ there is no need for additional measurements for the coefficient of $I$.

Since $X$ is a hermitian operator with eigenvalues $1$ and $-1$, its spectral decomposition is $X=1\cdot P^x_0 + (-1)\cdot P^x_1$ where $P^x_0=\frac{1}{2}\left(\begin{array}{cc}
1 & 1\\
1 & 1
\end{array}\right)$ is the projector into the eigenspace of 1 and $P^x_1=\frac{1}{2}\left(\begin{array}{cc}
1 & -1\\
-1 & 1
\end{array}\right)$ is the projector to the eigenspace of -1.

So we have $\text{tr}\left(X\rho\right)=\text{tr}\left(P_{0}^{x}\rho\right)-\text{tr}\left(P_{1}^{x}\rho\right)$.

By Born's rule, $\text{tr}\left(P_{i}^{z}\rho\right)$ is the probability for the outcome $\left|i\right\rangle$ when measuring in the X-basis, and this probability can be estimated directly using repeated meausrements in the X-basis. The $Y$ and $Z$ bases are handled
similarily.